In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import open3d as o3d
from torch.utils.data import DataLoader
from learning3d.data_utils import ClassificationData, ModelNet40Data
from SpinNet128 import Descriptor_Net

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ModelNet40 veri kümesini yükleme
train_dataset = ClassificationData(data_class=ModelNet40Data(train=True, num_points=4096))
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

test_dataset = ClassificationData(data_class=ModelNet40Data(train=False, num_points=4096))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

def compute_normals_single(points_single):
    """Tek bir nokta bulutu için Open3D ile normalleri hesaplar."""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_single.astype(np.float64))
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    return np.asarray(pcd.normals)

def augment(points):
    """Veri artırma işlemi uygular."""
    noise = torch.randn_like(points) * 0.03  # Gürültü ekle
    angles = torch.rand(3) * 2 * torch.pi  # 3 eksende rastgele döndür
    
    Rx = torch.tensor([[1, 0, 0],
                       [0, torch.cos(angles[0]), -torch.sin(angles[0])],
                       [0, torch.sin(angles[0]), torch.cos(angles[0])]])
    
    Ry = torch.tensor([[torch.cos(angles[1]), 0, torch.sin(angles[1])],
                       [0, 1, 0],
                       [-torch.sin(angles[1]), 0, torch.cos(angles[1])]])
    
    Rz = torch.tensor([[torch.cos(angles[2]), -torch.sin(angles[2]), 0],
                       [torch.sin(angles[2]), torch.cos(angles[2]), 0],
                       [0, 0, 1]])
    
    rotation_matrix = Rz @ Ry @ Rx
    rotation_matrix = rotation_matrix.to(points.device)
    points = torch.matmul(points, rotation_matrix)  # Döndür
    points = points + noise  # Gürültü ekle
    return points

class SpinNetClassifier(nn.Module):
    def __init__(self, num_classes=40, embedding_dim=128):
        super(SpinNetClassifier, self).__init__()
        self.encoder = self.encoder = Descriptor_Net(
            rad_n = 9,
            azi_n = 80,
            ele_n = 40,
            des_r = 0.30,
            voxel_r = 0.04,
            voxel_sample = 30,
            dataset = "3DMatch"
        )
        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.4),

            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.4),

            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(0.3),

            nn.Linear(512, num_classes)
        )
    
    def forward(self, points):
        features = self.encoder(points)
        global_embedding = features.squeeze()
        logits = self.fc(global_embedding)
        return logits

# Modeli oluştur
model = SpinNetClassifier(num_classes=40, embedding_dim=128).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999), weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6)

num_epochs = 30
train_losses, val_losses, train_accs, val_accs = [], [], [], []

for epoch in range(num_epochs):
    model.train()
    epoch_loss, correct, total = 0, 0, 0
    for i, data in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        points, labels = data
        points = (points - points.mean(dim=1, keepdim=True)) / torch.max(torch.norm(points, dim=-1, keepdim=True))
        points = points.to(device, dtype=torch.float32)
        labels = labels.squeeze().to(device)
        
        outputs = model(points)
        
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    
    train_losses.append(epoch_loss / len(train_loader))
    train_accs.append(100 * correct / total)
    
    # Validation
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for data in test_loader:
            points, labels = data
            points = points.to(device, dtype=torch.float32)
            labels = labels.squeeze().to(device)
            
            outputs = model(points)
            
            val_loss += criterion(outputs, labels).item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
    
    val_losses.append(val_loss / len(test_loader))
    val_accs.append(100 * correct / total)
    
    # Modeli kaydet
    torch.save(model.encoder.state_dict(), f"ppfnet_encoder_epoch{epoch+1}.pth")
    torch.save(model.fc.state_dict(), f"ppfnet_classifier_epoch{epoch+1}.pth")
    scheduler.step(val_losses[-1])  # ReduceLROnPlateau kullanılıyorsa
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accs[-1]:.2f}%, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accs[-1]:.2f}%")

# Sonuçları görselleştir
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs. Epochs')

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_accs, label='Train Accuracy')
plt.plot(range(1, num_epochs + 1), val_accs, label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Accuracy vs. Epochs')

plt.show()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using device: cuda


Epoch 1/30:  50%|██████████████              | 615/1230 [05:47<05:52,  1.75it/s]

In [ ]:
import matplotlib.pyplot as plt
# Sonuçları görselleştir
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs. Epochs')

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_accs, label='Train Accuracy')
plt.plot(range(1, num_epochs + 1), val_accs, label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Accuracy vs. Epochs')

plt.savefig("ppfnet_train.png", dpi=300)
plt.show()